In [9]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
#from category_encoders import OrdinalEncoder, OneHotEncoder

In [10]:
pd.options.display.max_rows = 30
pd.options.display.max_columns = 30sklearn.metrics

In [11]:
df = pd.read_csv('movie_data_training_set.csv')

In [12]:
condition = df['budget'] == 0

df.loc[condition, 'budget'] = df['budget'].mean()

df.drop(columns=['genre_list','pro_co'])

,id,adult,budget,original_language,popularity,revenue,runtime,status,video,vote_average,vote_count,country,Animation,Comedy,Family,...,Drama,Action,Crime,Thriller,Horror,History,Mystery,War,Foreign,Music,Documentary,Western,Aniplex,GoHands,score
0,862,False,3.000000e+07,en,21.946943,373554033.0,81.0,Released,False,7.7,5415.0,US,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.668945
1,8844,False,6.500000e+07,en,17.015539,262797249.0,104.0,Released,False,6.9,2413.0,US,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.857873
2,15602,False,2.406172e+07,en,11.7129,0.0,101.0,Released,False,6.5,92.0,US,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.084442
3,11862,False,2.406172e+07,en,8.387519000000001,76578911.0,106.0,Released,False,5.7,173.0,US,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.673698
4,949,False,6.000000e+07,en,17.924927,187436818.0,170.0,Released,False,7.7,1886.0,US,0,0,0,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,7.613259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6826,277839,False,5.402000e+06,fr,5.613875,0.0,100.0,Released,False,5.3,153.0,FR,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.411034
6827,430365,False,2.406172e+07,fr,7.250495,0.0,92.0,Released,False,5.2,94.0,FR,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.394847
6828,248705,False,2.586883e+07,fr,7.29492,0.0,110.0,Released,False,4.0,167.0,FR,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.532904
6829,455661,False,2.406172e+07,en,20.82178,0.0,4.0,Released,False,8.3,146.0,US,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7.335496


In [13]:
df[categorical].isna().sum()

NameError: name 'categorical' is not defined

In [14]:
df['status'] = df['status'].fillna(value='Released')

def convert_popularity(data):
    try:
        return float(data)
    except ValueError:
        print(data)
        return 0

df['popularity'] = df['popularity'].apply(convert_popularity)
df['popularity'] = df['popularity'].fillna(method='ffill')
df['video'] = df['video'].fillna(method='ffill')
df['country'] = df['country'].fillna(value='US')

Beware Of Frost Bites


In [15]:
categorical = [col for col in df.select_dtypes('object').columns]
print(categorical)
numerical = []
for columns in df.columns:
    if columns in categorical:
        pass
    else:
        numerical.append(columns)
numerical.remove('score')
print(numerical)

['id', 'original_language', 'status', 'country', 'genre_list', 'pro_co']
['adult', 'budget', 'popularity', 'revenue', 'runtime', 'video', 'vote_average', 'vote_count', 'Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'Aniplex', 'GoHands']


In [16]:
# try:
#     float(element)
# except ValueError:
#     print "Not a float"
############################################
# SPLIT DATA INTO FEATURES AND TARGET SETS #
############################################
y = df['score']
X = df.drop(columns=['score','vote_average','vote_count'])


In [18]:
prep_numericals = Pipeline([
    ('simpleimputer', SimpleImputer()),
    ('standardscaler', StandardScaler())
])

prep_categoricals = Pipeline([
    ('onehotencoder', OneHotEncoder()),
    
])

full_pipeline = ColumnTransformer([
    ('num', prep_numericals, numerical),
    ('cat', OrdinalEncoder(), categorical)
    ])

second_pipeline = ColumnTransformer([
    ('num', prep_numericals, numerical),
    ('cat', OneHotEncoder(), categorical)
    ])

movie_data_prepared = full_pipeline.fit_transform(df)
# movie_data_ohe = second_pipeline.fit_transform(df)


X_train, X_val, y_train, y_val = train_test_split(movie_data_prepared, y, train_size=0.8, random_state=10)
# X_train_ohe, X_val_ohe, y_train_ohe, y_val_ohe = 
#     train_test_split(movie_data_ohe, y, train_size=0.8, random_state=10)

In [5]:
model_lr = LinearRegression()
#model_lr_ohe = LinearRegression()
model_dtr = DecisionTreeRegressor()

In [19]:

model_lr.fit(X_train,y_train)
# model_lr_ohe.fit(X_train_ohe, y_train,ohe)

LinearRegression()

In [35]:
# Defining my baseline as the mse associated with constant prediction of the mean of 'score' #
y_train_mean = [y_train.mean()] * len(y_train)
baseline = mean_squared_error(y_train, y_train_mean, squared=False)
print(baseline)

0.6835361649981446


In [36]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model_lr, X_train, y_train, cv=3)

In [37]:
print(cv_results)

{'fit_time': array([0.00718093, 0.00553298, 0.00417233]), 'score_time': array([0.00087023, 0.00082707, 0.00056243]), 'test_score': array([0.95734634, 0.95725384, 0.96029068])}


In [38]:
y_pred_val = model_lr.predict(X_val)

In [39]:
rmse = mean_squared_error(y_val, y_pred_val, squared=False)

In [40]:
print('Root Mean Squared Error for Linear Regression: ',rmse)

Root Mean Squared Error for Linear Regression:  0.13712486954200814


In [31]:
#zipped = list(zip(y_val,y_pred_val))

In [30]:
#print(zipped)

In [41]:
model_dtr.fit(X_train, y_train)
y_pred_val = model_dtr.predict(X_val)
rmse_dtr = mean_squared_error(y_val, y_pred_val, squared=False)
print('Root Mean Squared Error for DecisionTreeRegressor: ',rmse_dtr)

Root Mean Squared Error for DecisionTreeRegressor:  0.0283709350685237
